In [ ]:
import json

def merge_json_strings(json_strings):
    merged_data = {"paragraphs": []}
    
    for json_str in json_strings:
        data = json.loads(json_str)
        
        if 'paragraphs' in data:
            merged_data['paragraphs'].extend(data['paragraphs'])
    
    return merged_data

# Ví dụ sử dụng:
json_strings = [
    '{"paragraphs": [{"text": "Xin chào", "level": "1"}]}',
    '{"paragraphs": [{"text": "Chào mừng", "level": "2"}]}',
    '{"paragraphs": [{"text": "Hẹn gặp lại", "level": "1"}]}'
]

result = merge_json_strings(json_strings)

# In kết quả
print(json.dumps(result, indent=4, ensure_ascii=False))

print("Đã gộp xong chuỗi JSON!")


In [ ]:
import PyPDF2
import google.generativeai as genai

# Khởi tạo API Gemini

genai.configure(api_key="")


def read_pdf(file_path):
    """Đọc nội dung từ file PDF"""
    content = ""
    with open(file_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            content += page.extract_text()
    return content


def split_text(text, chunk_size=3000):
    """Chia văn bản thành các đoạn nhỏ"""
    chunks = []
    for i in range(0, len(text), chunk_size):
        chunks.append(text[i : i + chunk_size])
    return chunks


def clean_and_summarize_with_gemini(text):
    """Làm sạch và tóm tắt nội dung văn bản bằng Gemini"""
    model = genai.GenerativeModel("gemini-2.0-flash")
    schema = """
    {
"paragraphs: [
{"text":"...", "level":"..."}
]
}
    """
    prompt = f"""Làm sạch nội dung sau và tóm tắt thành các bullet point cho bài thuyết trình theo từng đề mục, 
        kết quả dưới dạng JSON:
        {schema}
        trong đó level là các giá trị như: heading 1, heading 2, heading 3, normal
        Chỉ đưa ra kết quả, không giải thích hay nói gì thêm:\n{text}"""
    response = model.generate_content(prompt)
    return response.text.replace("```json","").replace("```","")


def example():
    pdf_path = "D:/example.pdf"  # Thay bằng đường dẫn file PDF của bạn
    raw_text = read_pdf(pdf_path)
    
    # Chia nhỏ nội dung nếu quá dài
    text_chunks = split_text(raw_text)
    summarized_chunks = []
    
    # Làm sạch và tóm tắt từng đoạn rồi tổng hợp lại
    for i, chunk in enumerate(text_chunks):
        print(f"Đang làm sạch và tóm tắt đoạn {i + 1}/{len(text_chunks)}...")
        summary_chunk = clean_and_summarize_with_gemini(chunk)
        summarized_chunks.append(summary_chunk)
    
    # Ghép nội dung đã làm sạch và tóm tắt
    summarized_text = merge_json_strings(summarized_chunks)
    
    # In nội dung đã làm sạch và tóm tắt
    print(summarized_text)


    print("Nội dung đã được làm sạch, tóm tắt và lưu vào 'cleaned_and_summarized_text.txt'")



In [ ]:
example()